In [143]:
import os
import pandas as pd
import json
import csv
print(os.listdir("."))

['label.json', '.ipynb_checkpoints', 'SlotFilling.ipynb', 'slot.csv', 'data.json']


# Slots' name in our data

In [136]:
from enum import Enum

class Label(Enum):
    B_2param  = 0
    B_2param2 = 1
    B_3param  = 2
    B_param   = 3
    B_param2  = 4
    B_param3  = 5
    I_2param  = 6
    I_2param2 = 7
    I_3param  = 8
    I_param   = 9
    I_param2  = 10
    O         = 11


Read data

In [160]:
with open('data.json') as f:
    data_query = json.load(f)
    
with open('label.json') as f:
    data_slots = json.load(f)

Get index from our data (question number)

In [195]:
keys = list(data_slots.keys()) # is same as data_query.keys()

In [317]:
"""
make_slot : Make the data which contain slots (slot name)
@query : The labels we set.
@label : Each problem has different labels. Therefore, Fetch the label which is set for each questions.
@slots : Using class Label, replace query to slots
"""

def make_slot():
    for key in keys:
        query = [] # we assigned before per question 
        label = data_slots[key] # label in each question
        slots = []  # integrate label : the output

        for l in data_query[key]: # l : label
            int_label = int(l[1]) # if it is "O"
            if(int_label == 0):
                slots.append(Label.O.value)
                #query.append(int(l[1]))
            else:                 # if it is not "O", such as "B" and "I"
                label_name = label[int_label-1]
                label_name = label_name.replace("-", "_") # we have "-" in string, so change it as "_"
                slots.append(Label[label_name].value)

                #query.append(int(l[1]))

        #print(query)
        print(slots)
        print(label)

        # save slots in csv
        with open('slots.csv', 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(slots)
        csvFile.close()

        print("\n")
        

    
"""
make_query : Make the data which contain query, the word vector
@query : It is an output. It takes the query of question(word vector)
@label : Each problem has different labels. Therefore, Fetch the label which is set for each questions.
"""

def make_query():
    voca = make_voca()

    for key in keys:
        index_BOS = voca.index("<BOS>")
        query = [index_BOS] # we assigned before per question 

        for l in data_query[key]: # l : label
            # if it is "O"
            print(l[0])
            index = voca.index(l[0])
            query.append(index)

        index_EOS = voca.index("<EOS>") 
        query.append(index_EOS)
        print(query)

        # save slots in csv
        with open('query.csv', 'a') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(query)
        csvFile.close()

        print("\n")
        
"""
make_voca : To make vocabulary, collect all words, remove duplicates, and create vocabulary set. 
"""    
def make_voca():
    voca = []
    for key in keys:
        words = data_query.values()

        for i in data_query[key]:
            word = i[0]
            voca.append(word)

    voca = list(set(voca))
    voca.sort(reverse = False) 
    
    #add BOS (Beginning of Sentense) and EOS (End of Sentense) 
    voca.append("<BOS>")
    voca.append("<EOS>")
    

    #save voca in csv
    with open('voca.csv', 'a') as csvFile:
        writer = csv.writer(csvFile, delimiter=',')
        for v in voca:
            writer.writerow([v])
    csvFile.close()
    
    
    return voca